In [107]:
import os
import json
from dotenv import load_dotenv
import folium
import pandas as pd
import geopandas as gpd
from folium.features import GeoJsonTooltip
from folium.plugins import MarkerCluster, Draw, Fullscreen, Geocoder, HeatMap, MiniMap, MeasureControl, MousePosition, SideBySideLayers, DualMap
from preprocessing_func import * 

load_dotenv()

True

Create a text file named `.env` and place your Mapbox API key with the following format:
```
MAPBOX_API_KEY="your_api_key"
```	
Don't forget to add this file to your `.gitignore` (you don't want to share your API key with the world!)

In [109]:
MAPBOX_API_KEY = os.getenv('MAPBOX_API_KEY')

## Data Processing

You can check how the datasets are created in [the preprocessing functions script](preprocessing_func.py). The clean datasets are created by running the following commands:
```

In [2]:
points_gdf = process_places("data/CB_points_of_interest.gpkg")
polygons_gdf = process_zones("data/CB_IMD_2019.geojson")
choropleth_gdf = create_choropleth_data(points_gdf, polygons_gdf)

c:\Users\us\anaconda3\envs\detectree2-env\lib\site-packages\IPython\core\interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


You can also read the files in `data/` that are cleaned and ready to use (*i.e.* `/*_clean.geojson`).

In [145]:
data_class = 'cafe'
datasets = filter_datasets(points_gdf, choropleth_gdf)
cafe_markers_gdf = datasets[f"{data_class}_gdf"]
bar_map_gdf = datasets[f"{data_class}_choropleth_gdf"]

data_class = 'bar'
datasets = filter_datasets(points_gdf, choropleth_gdf)
bar_markers_gdf = datasets[f"{data_class}_gdf"]
cafe_map_gdf = datasets[f"{data_class}_choropleth_gdf"]

## Standard Map

In [152]:
# Create map
m = folium.Map(location=[52.205320488806095, 0.11825440531707056], zoom_start=15, tiles='cartodbpositron')
m

## Basemaps

In [153]:
# Add custom base maps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = False,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = False,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite Hybrid',
        overlay = False,
        control = True
    ),
    'Vintage': folium.TileLayer(
        tiles = 'https://api.mapbox.com/styles/v1/acz25/clopmmaqa00ge01nzeuv38zfb/tiles/256/{z}/{x}/{y}@2x?access_token=' + MAPBOX_API_KEY,
        attr = 'Mapbox',
        name = 'Vintage',
        overlay = False,
        control = True
    ),
}

basemaps['Google Satellite Hybrid'].add_to(m)
basemaps['Google Satellite'].add_to(m)
basemaps['Google Maps'].add_to(m)
folium.LayerControl().add_to(m)
m

In [150]:
def create_basemap():
    
    m = folium.Map(location=[52.205320488806095, 0.11825440531707056], zoom_start=15, tiles='cartodbpositron')
    
    basemaps['Google Satellite Hybrid'].add_to(m)
    basemaps['Google Satellite'].add_to(m)
    basemaps['Google Maps'].add_to(m)
    
    return m

## Choropleth Map

In [154]:
m = create_basemap()

data_class = 'cafe'

# Create choropleth
choropleth = folium.Choropleth(
    geo_data=cafe_map_gdf,
    name=f'Choropleth ({data_class})',
    data=cafe_map_gdf,
    columns=['lsoa11nmw', 'count'],
    key_on='feature.properties.lsoa11nmw',
    fill_color='YlGnBu',
    fill_opacity=1,
    line_opacity=0.5,
    legend_name='Counts'
)

# Add tooltip to the choropleth
choropleth.geojson.add_child(
    GeoJsonTooltip(
        fields=['classname', 'lsoa11nmw', 'count'],
        aliases=['Type: ', 'Area: ', 'Count: '],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
)

# Add choropleth to the map
choropleth.add_to(m)

folium.LayerControl().add_to(m)
m

## Markers

In [155]:
m = create_basemap()

# Feature group for points
points_feature_group = folium.FeatureGroup(name=f'Markers ({data_class})').add_to(m)

# Add points to the feature group
for feature in json.loads(bar_markers_gdf.to_json())['features']:
    classname = feature['properties']['classname']
    name = feature['properties']['name']
    address = feature['properties']['street_name']
    url = feature['properties']['url']
    url = '' if url is None else url
    geom = feature['geometry']['coordinates']
    
    marker = folium.Marker(
        location=[geom[1], geom[0]],
        popup=folium.Popup(html=create_popup_html(name, address, url), max_width=300),
        tooltip=folium.Tooltip(text=create_tooltip_html(name)),
        icon=folium.Icon(color='orange', icon='coffee', prefix='fa') if classname == 'Cafes, Snack Bars and Tea Rooms' else folium.Icon(color='darkblue', icon='glass')
    )
    marker.add_to(points_feature_group)
    
folium.LayerControl().add_to(m)
m

### Marker Cluster

In [156]:
m = create_basemap()

marker_cluster = MarkerCluster(name=f'Markers ({data_class})').add_to(m)

# Feature group for points
# points_feature_group = folium.FeatureGroup(name=f'Markers ({data_class})').add_to(m)

# Add points to the feature group
for feature in json.loads(bar_markers_gdf.to_json())['features']:
    classname = feature['properties']['classname']
    name = feature['properties']['name']
    address = feature['properties']['street_name']
    url = feature['properties']['url']
    url = '' if url is None else url
    geom = feature['geometry']['coordinates']
    
    marker = folium.Marker(
        location=[geom[1], geom[0]],
        popup=folium.Popup(html=create_popup_html(name, address, url), max_width=300),
        tooltip=folium.Tooltip(text=create_tooltip_html(name)),
        icon=folium.Icon(color='orange', icon='coffee', prefix='fa') if classname == 'Cafes, Snack Bars and Tea Rooms' else folium.Icon(color='darkblue', icon='glass')
    )
    marker.add_to(marker_cluster)
    
folium.LayerControl().add_to(m)
m

## Plugins

### Layer Control

In [159]:
m = create_basemap()

MiniMap().add_to(m)

Draw(export=True).add_to(m)

Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

Geocoder(position='topcenter').add_to(m)

# MeasureControl().add_child(m)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' &deg; ';};"

MousePosition(
    position="bottomleft",
    separator=" | ",
    empty_string="NaN",
    lng_first=True,
    num_digits=20,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m)

m

### Dual Display

In [160]:
m = DualMap(location=[52.205320488806095, 0.11825440531707056], tiles=None, zoom_start=15)

basemaps['Vintage'].add_to(m.m1)
folium.TileLayer("openstreetmap").add_to(m.m2)

data_class = 'bar'
bar_cluster = MarkerCluster(name=f'Markers ({data_class})').add_to(m.m1)

# Add points to the feature group
for feature in json.loads(bar_markers_gdf.to_json())['features']:
    classname = feature['properties']['classname']
    name = feature['properties']['name']
    address = feature['properties']['street_name']
    url = feature['properties']['url']
    url = '' if url is None else url
    geom = feature['geometry']['coordinates']
    
    marker = folium.Marker(
        location=[geom[1], geom[0]],
        popup=folium.Popup(html=create_popup_html(name, address, url), max_width=300),
        tooltip=folium.Tooltip(text=create_tooltip_html(name)),
        icon=folium.Icon(color='orange', icon='coffee', prefix='fa') if classname == 'Cafes, Snack Bars and Tea Rooms' else folium.Icon(color='darkblue', icon='glass')
    )
    marker.add_to(bar_cluster)
    
data_class = 'cafe'
cafe_cluster = MarkerCluster(name=f'Markers ({data_class})').add_to(m.m2)

# Add points to the feature group
for feature in json.loads(cafe_markers_gdf.to_json())['features']:
    classname = feature['properties']['classname']
    name = feature['properties']['name']
    address = feature['properties']['street_name']
    url = feature['properties']['url']
    url = '' if url is None else url
    geom = feature['geometry']['coordinates']
    
    marker = folium.Marker(
        location=[geom[1], geom[0]],
        popup=folium.Popup(html=create_popup_html(name, address, url), max_width=300),
        tooltip=folium.Tooltip(text=create_tooltip_html(name)),
        icon=folium.Icon(color='orange', icon='coffee', prefix='fa') if classname == 'Cafes, Snack Bars and Tea Rooms' else folium.Icon(color='darkblue', icon='glass')
    )
    marker.add_to(cafe_cluster)
    
folium.LayerControl().add_to(m)
m

### Heatmap

In [168]:
m = DualMap(location=[52.205320488806095, 0.11825440531707056], tiles=None, zoom_start=15)

basemaps['Vintage'].add_to(m.m1)
folium.TileLayer("openstreetmap").add_to(m.m2)
data_class = 'bar'
bar_cluster = folium.FeatureGroup(name=f'Heatmap ({data_class})').add_to(m.m1)

HeatMap([[point.y, point.x] for point in bar_markers_gdf.geometry]).add_to(bar_cluster)
data_class = 'cafe'
cafe_cluster = folium.FeatureGroup(name=f'Heatmap ({data_class})').add_to(m.m2)

HeatMap([[point.y, point.x] for point in cafe_markers_gdf.geometry]).add_to(cafe_cluster)
 
folium.LayerControl().add_to(m)
m

## Publishing (GitHub Pages)

In [169]:
m.save('index.html')

Enable Pages in the repo configuration and select the `main` branch as the source. If you use another branch, you can select it in the dropdown menu.